---
description: Cookbook that showcases Opik's integration with the CrewAI
---

# Using Opik with CrewAI

This notebook showcases how to use Opik with CrewAI. [CrewAI](https://github.com/crewAIInc/crewAI) is a cutting-edge framework for orchestrating autonomous AI agents.
> CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks.

> Think of it as assembling your dream team - each member (agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives.

For this guide we will use CrewAI's quickstart example.

## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=llamaindex&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&=opik&utm_medium=colab&utm_content=llamaindex&utm_campaign=opik) and grab you API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik&utm_medium=colab&utm_content=llamaindex&utm_campaign=opik) for more information.

In [ ]:
%pip install crewai crewai-tools opik --upgrade

In [ ]:
import opik

opik.configure(use_local=False)

## Preparing our environment

First, we set up our API keys for our LLM-provider as environment variables:

In [1]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Using CrewAI

Before we create `crew` object and run it we should create two files: `agents.yaml` and `tasks.yaml`

In [ ]:
# Ensure the directory exists
directory = "config"
os.makedirs(directory, exist_ok=True)

In [ ]:
import os
import yaml

# Define the content
content = {
    "researcher": {
        "role": "{topic} Senior Data Researcher",
        "goal": "Uncover cutting-edge developments in {topic}",
        "backstory": (
            "You're a seasoned researcher with a knack for uncovering the latest "
            "developments in {topic}. Known for your ability to find the most relevant "
            "information and present it in a clear and concise manner."
        ),
    },
    "reporting_analyst": {
        "role": "{topic} Reporting Analyst",
        "goal": "Create detailed reports based on {topic} data analysis and research findings",
        "backstory": (
            "You're a meticulous analyst with a keen eye for detail. You're known for "
            "your ability to turn complex data into clear and concise reports, making "
            "it easy for others to understand and act on the information you provide."
        ),
    },
}

file_path = os.path.join(directory, "agents.yaml")

with open(file_path, "w") as file:
    yaml.dump(content, file, default_flow_style=False)


In [ ]:
content = {
    "research_task": {
        "description": (
            "Conduct a thorough research about {topic} "
            "Make sure you find any interesting and relevant information given "
            "the current year is 2024."
        ),
        "expected_output": "A list with 5 bullet points of the most relevant information about {topic}",
        "agent": "researcher",
    },
    "reporting_task": {
        "description": (
            "Review the context you got and expand each topic into a full section for a report. "
            "Make sure the report is detailed and contains any and all relevant information."
        ),
        "expected_output": (
            "A fully fledge reports with the mains topics, each with a full section of information. "
            "Formatted as markdown without '```'"
        ),
        "agent": "reporting_analyst",
        "output_file": "report.md",
    },
}

file_path = os.path.join(directory, "tasks.yaml")

with open(file_path, "w") as file:
    yaml.dump(content, file, default_flow_style=False)


The next step is to create our `crew` object:

In [2]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task


@CrewBase
class LatestAiDevelopmentCrew:
    """LatestAiDevelopment crew"""

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'],
            verbose=True,
        )

    @agent
    def reporting_analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['reporting_analyst'],
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task'],
        )

    @task
    def reporting_task(self) -> Task:
        return Task(
            config=self.tasks_config['reporting_task'],
            output_file='output/report.md'  # This is the file that will contain the final report.
        )

    @crew
    def crew(self) -> Crew:
        """Creates the LatestAiDevelopment crew"""
        return Crew(
            agents=self.agents,  # Automatically created by the @agent decorator
            tasks=self.tasks,  # Automatically created by the @task decorator
            process=Process.sequential,
            verbose=True,
        )


C:\Users\alexanderb\PycharmProjects\opik\.venv3.10\lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


OSError: source code not available

Now we can import Opik's tracker and run our crew:

In [ ]:
from opik.integrations.crewai import track_crewai

track_crewai(project_name="crewai-integration-demo")

inputs = {
    'topic': 'AI Agents'
}
my_crew = LatestAiDevelopmentCrew()
my_crew = my_crew.crew()
result = my_crew.kickoff(inputs=inputs)

print(result)

You can now go to the Opik app to see the trace:

![LlamaIndex trace in Opik](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/cookbook/llamaIndex_cookbook.png)